In [3]:
#pip install tqdm

In [4]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd 
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# URL con la API de clima 

In [5]:
query = 'Valencia/Spain'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=APIKEY&q=Valencia/Spain&days=1&aqi=no&alerts=no'

In [6]:
response = requests.get(url_clima).json()

In [7]:
response

{'location': {'name': 'Valencia',
  'region': 'Comunidad Valenciana',
  'country': 'Spain',
  'lat': 39.47,
  'lon': -0.37,
  'tz_id': 'Europe/Madrid',
  'localtime_epoch': 1715509573,
  'localtime': '2024-05-12 12:26'},
 'current': {'last_updated_epoch': 1715508900,
  'last_updated': '2024-05-12 12:15',
  'temp_c': 23.0,
  'temp_f': 73.4,
  'is_day': 1,
  'condition': {'text': 'Sunny',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
   'code': 1000},
  'wind_mph': 6.9,
  'wind_kph': 11.2,
  'wind_degree': 100,
  'wind_dir': 'E',
  'pressure_mb': 1017.0,
  'pressure_in': 30.03,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 53,
  'cloud': 0,
  'feelslike_c': 24.9,
  'feelslike_f': 76.8,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 6.0,
  'gust_mph': 13.4,
  'gust_kph': 21.5},
 'forecast': {'forecastday': [{'date': '2024-05-12',
    'date_epoch': 1715472000,
    'day': {'maxtemp_c': 21.7,
     'maxtemp_f': 71.1,
     'mintemp_c': 15.6,
     'mintemp_f': 60.0,
     '

In [8]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [9]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [10]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [11]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1715464800,
 'time': '2024-05-12 00:00',
 'temp_c': 16.4,
 'temp_f': 61.5,
 'is_day': 0,
 'condition': {'text': 'Patchy rain nearby',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/176.png',
  'code': 1063},
 'wind_mph': 3.6,
 'wind_kph': 5.8,
 'wind_degree': 320,
 'wind_dir': 'NW',
 'pressure_mb': 1017.0,
 'pressure_in': 30.04,
 'precip_mm': 0.04,
 'precip_in': 0.0,
 'snow_cm': 0.0,
 'humidity': 82,
 'cloud': 79,
 'feelslike_c': 16.4,
 'feelslike_f': 61.5,
 'windchill_c': 16.4,
 'windchill_f': 61.5,
 'heatindex_c': 16.4,
 'heatindex_f': 61.5,
 'dewpoint_c': 13.3,
 'dewpoint_f': 55.9,
 'will_it_rain': 1,
 'chance_of_rain': 85,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 6.2,
 'gust_kph': 10.0,
 'uv': 1.0}

In [12]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

'2024-05-12'

In [13]:
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0])

1

In [14]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']

'Patchy rain nearby'

In [15]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

16.4

In [16]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

1

In [17]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

84

# Dataframe

In [18]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [19]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████| 24/24 [00:00<?, ?it/s]

In [20]:
datos[0]

('2024-05-12', 0, 'Patchy rain nearby', 16.4, 1, 85)

In [21]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2024-05-12,0,Patchy rain nearby,16.4,1,85
1,2024-05-12,1,Patchy rain nearby,16.1,1,100
2,2024-05-12,2,Patchy rain nearby,15.6,1,84
3,2024-05-12,3,Clear,15.6,0,0
4,2024-05-12,4,Clear,15.6,0,0
5,2024-05-12,5,Clear,15.6,0,0
6,2024-05-12,6,Clear,16.6,0,0
7,2024-05-12,7,Sunny,17.9,0,0
8,2024-05-12,8,Sunny,18.8,0,0
9,2024-05-12,9,Sunny,19.7,0,0


In [22]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [23]:
df[df['Lluvia']==1]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2024-05-12,0,Patchy rain nearby,16.4,1,85
1,2024-05-12,1,Patchy rain nearby,16.1,1,100
2,2024-05-12,2,Patchy rain nearby,15.6,1,84


In [24]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion','prob_lluvia']]
df_rain.set_index('Hora', inplace = True)

In [25]:
df['prob_lluvia'][0]

85

In [26]:
df_rain

,Condicion,prob_lluvia
Hora,,


In [27]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2024-05-12 en Valencia/Spain es : \n\n\n Empty DataFrame\nColumns: [Condicion, prob_lluvia]\nIndex: []'

In [28]:
TWILIO_PHONE_NUMBER

'+18482533810'

# Mensaje SMS desde Twilio

In [29]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico del clima de hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=TWILIO_PHONE_NUMBER,
                     to=USER_PHONE
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SM8694c6dd5a1cc258254f552dbd48d077


# Mensaje por WhatsApp desde Twilio

In [30]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_='Whatsapp:' + TWILIO_WHATSAPP_NUMBER,
                     to='Whatsapp:' + USER_PHONE
                 )

print('Mensaje de WhatsApp Enviado ' + message.sid)

Mensaje de WhatsApp Enviado SM5f21c506aab8c12ed716ab006f3f3527
